In [40]:
import pandas as pd
import os
from transformers import GPT2TokenizerFast, GPT2LMHeadModel, pipeline
import math
import torch
import numpy as np

input_dir = '../data/inputs'
output_dir = '../data/outputs/s0'
results_dir = '../data/results'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

In [41]:
model_str_map = {
    'llama3-8b': 'Llama3 8B',
    'llama3-8b-instruct': 'Llama3 8B (Instruct)',
    'mistral-7b': 'Mistral 7B',
    'mistral-7b-instruct': 'Mistral 7B (Instruct)',
    'llama2-7b': 'Llama2 7B',
    'llama2-7b-instruct': 'Llama2 7B (Instruct)',
}
model_strs = ['llama3-8b', 'llama3-8b-instruct', 'mistral-7b', 'mistral-7b-instruct', 'llama2-7b', 'llama2-7b-instruct']
model_strs = sorted(model_strs, key=len, reverse=True)

In [42]:
female_ratios = pd.read_csv(os.path.join(input_dir, 'female_ratios.csv'))
# for model in  llama3-8b llama3-8b-instruct mistral-7b mistral-7b-instruct llama2-7b llama2-7b-chat


prompt_ids = ['none','low-1','low-2','medium-3','medium-4','high-5','high-6']
prompt_id_mapping = {pid: idx for idx, pid in enumerate(prompt_ids)}

df_list = []
for f in os.listdir(output_dir):
    if "conv" not in f and "gender" not in f and 'gpt2' not in f:
    # if 'conversation.csv' in f:
        
        df = pd.read_csv(os.path.join(output_dir, f))
    # if 'conversation.csv' in f:
        df = pd.read_csv(os.path.join(output_dir, f))
        df = pd.merge(df,female_ratios,on='job')
        df = df.drop(columns=['job','Unnamed: 0'])

        df['female_dominated'] = df['female_ratio'] > 50
        # Extract prompt ID from filename
        prompt_id_str = next((pid for pid in prompt_ids if pid in f), 'none')
        prompt_id = prompt_id_mapping[prompt_id_str]
        df['debiasing_prompt_id'] = prompt_id
        
                # Extract model from filename
        model_str = next((model for model in model_strs if model in f), None)
        df['model'] = model_str
        df['conversation'] = 'conv' in f

        # Remove model name from other column names
        if model_str:
            df = df.rename(columns=lambda x: x.replace(f'{model_str}_', '') if model_str in x else x)

        
        # grouped_df = df.groupby('female_dominated').mean().reset_index()
        numeric_cols = df.select_dtypes(include='number').columns
        grouped_df = df.groupby(['female_dominated', 'model','conversation'])[numeric_cols].mean().reset_index()
    
        if 'male_met-met' in df.columns:
            col_ends = ['met-met', 'friend', 'talk-met']
            # Compute averages for male, female, and diverse columns
            male_cols = ['male_' + end for end in col_ends]
            female_cols = ['female_' + end for end in col_ends]
            diverse_cols = ['diverse_' + end for end in col_ends]

    
            grouped_df['male'] = grouped_df[male_cols].mean(axis=1)
            grouped_df['female'] = grouped_df[female_cols].mean(axis=1)
            grouped_df['diverse'] = grouped_df[diverse_cols].mean(axis=1)
            # make sure the three columns add up to 1
            # grouped_df['total'] = grouped_df['male'] + grouped_df['female'] + grouped_df['diverse']
            # 
            # Optionally, drop the original detailed columns if only averages are needed
            grouped_df = grouped_df.drop(columns=male_cols + female_cols + diverse_cols)

            
        df_list.append(grouped_df)
    # concat all the dataframes
df = pd.concat(df_list)
df.to_csv(os.path.join(results_dir, 'implicit.csv'), index=False)
        

In [43]:
# df_list = []
# for f in os.listdir(output_dir):
#     if 'conversation.csv' in f:
#         df = pd.read_csv(os.path.join(output_dir, f))
#         df = pd.merge(df,female_ratios,on='job')
#         df = df.drop(columns=['job','Unnamed: 0'])
# 
#         df['female_dominated'] = df['female_ratio'] > 50
#         # Extract prompt ID from filename
#         prompt_id_str = next((pid for pid in prompt_ids if pid in f), 'none')
#         prompt_id = prompt_id_mapping[prompt_id_str]
#         df['debiasing_prompt_id'] = prompt_id
#         
#                 # Extract model from filename
#         model_str = next((model for model in model_strs if model in f), None)
#         df['model'] = model_str
#         df['conversation'] = 'conv' in f
# 
#         # Remove model name from other column names
#         if model_str:
#             df = df.rename(columns=lambda x: x.replace(f'{model_str}_', '') if model_str in x else x)
# 
#         
#         # grouped_df = df.groupby('female_dominated').mean().reset_index()
#         numeric_cols = df.select_dtypes(include='number').columns
#         grouped_df = df.groupby(['female_dominated', 'model','conversation'])[numeric_cols].mean().reset_index()
#     
#         if 'male_met-met' in df.columns:
#             col_ends = ['met-met', 'friend', 'talk-met']
#             # Compute averages for male, female, and diverse columns
#             male_cols = ['male_' + end for end in col_ends]
#             female_cols = ['female_' + end for end in col_ends]
#             diverse_cols = ['diverse_' + end for end in col_ends]
# 
#     
#             grouped_df['male'] = grouped_df[male_cols].mean(axis=1)
#             grouped_df['female'] = grouped_df[female_cols].mean(axis=1)
#             grouped_df['diverse'] = grouped_df[diverse_cols].mean(axis=1)
#             # make sure the three columns add up to 1
#             # grouped_df['total'] = grouped_df['male'] + grouped_df['female'] + grouped_df['diverse']
#             # 
#             # Optionally, drop the original detailed columns if only averages are needed
#             grouped_df = grouped_df.drop(columns=male_cols + female_cols + diverse_cols)
# 
#             
#         df_list.append(grouped_df)
#     # concat all the dataframes
# df = pd.concat(df_list)
# df.to_csv(os.path.join(results_dir, 'conversation.csv'), index=False)
#     

In [44]:
df_list = []
for f in os.listdir(output_dir):
    if 'genderquestion.csv' in f and 'gpt2' not in f:
        df = pd.read_csv(os.path.join(output_dir, f))
        df = pd.merge(df,female_ratios,on='job')
        df = df.drop(columns=['job','Unnamed: 0'])


        df['female_dominated'] = df['female_ratio'] > 50
        # Extract prompt ID from filename
        prompt_id_str = next((pid for pid in prompt_ids if pid in f), 'none')
        prompt_id = prompt_id_mapping[prompt_id_str]
        df['debiasing_prompt_id'] = prompt_id
        
                # Extract model from filename
        model_str = next((model for model in model_strs if model in f), None)
        df['model'] = model_str
        df['conversation'] = 'conv' in f

        # Remove model name from other column names
        if model_str:
            df = df.rename(columns=lambda x: x.replace(f'{model_str}_', '') if model_str in x else x)

        
        # grouped_df = df.groupby('female_dominated').mean().reset_index()
        numeric_cols = df.select_dtypes(include='number').columns
        grouped_df = df.groupby(['female_dominated', 'model','conversation'])[numeric_cols].mean().reset_index()
    
        if 'male_met-met' in df.columns:
            col_ends = ['met-met', 'friend', 'talk-met']
            # Compute averages for male, female, and diverse columns
            male_cols = ['male_' + end for end in col_ends]
            female_cols = ['female_' + end for end in col_ends]
            diverse_cols = ['diverse_' + end for end in col_ends]

    
            grouped_df['male'] = grouped_df[male_cols].mean(axis=1)
            grouped_df['female'] = grouped_df[female_cols].mean(axis=1)
            grouped_df['diverse'] = grouped_df[diverse_cols].mean(axis=1)
            # make sure the three columns add up to 1
            # grouped_df['total'] = grouped_df['male'] + grouped_df['female'] + grouped_df['diverse']
            # 
            # Optionally, drop the original detailed columns if only averages are needed
            grouped_df = grouped_df.drop(columns=male_cols + female_cols + diverse_cols)

            
        df_list.append(grouped_df)
    # concat all the dataframes
df = pd.concat(df_list)
df.to_csv(os.path.join(results_dir, 'genderquestion.csv'), index=False)

In [45]:
# df_list = []
# for f in os.listdir(output_dir):
#     if 'genderquestion_conv.csv' in f:
#         df = pd.read_csv(os.path.join(output_dir, f))
#         df = pd.merge(df,female_ratios,on='job')
#         df = df.drop(columns=['job','Unnamed: 0'])
#          
# 
#         df['female_dominated'] = df['female_ratio'] > 50
#         # Extract prompt ID from filename
#         prompt_id_str = next((pid for pid in prompt_ids if pid in f), 'none')
#         prompt_id = prompt_id_mapping[prompt_id_str]
#         df['debiasing_prompt_id'] = prompt_id
#         
#                 # Extract model from filename
#         model_str = next((model for model in model_strs if model in f), None)
#         df['model'] = model_str
#         df['conversation'] = 'conv' in f
# 
#         # Remove model name from other column names
#         if model_str:
#             df = df.rename(columns=lambda x: x.replace(f'{model_str}_', '') if model_str in x else x)
# 
#         
#         # grouped_df = df.groupby('female_dominated').mean().reset_index()
#         numeric_cols = df.select_dtypes(include='number').columns
#         grouped_df = df.groupby(['female_dominated', 'model','conversation'])[numeric_cols].mean().reset_index()
#     
#         if 'male_met-met' in df.columns:
#             col_ends = ['met-met', 'friend', 'talk-met']
#             # Compute averages for male, female, and diverse columns
#             male_cols = ['male_' + end for end in col_ends]
#             female_cols = ['female_' + end for end in col_ends]
#             diverse_cols = ['diverse_' + end for end in col_ends]
# 
#     
#             grouped_df['male'] = grouped_df[male_cols].mean(axis=1)
#             grouped_df['female'] = grouped_df[female_cols].mean(axis=1)
#             grouped_df['diverse'] = grouped_df[diverse_cols].mean(axis=1)
#             # make sure the three columns add up to 1
#             # grouped_df['total'] = grouped_df['male'] + grouped_df['female'] + grouped_df['diverse']
#             # 
#             # Optionally, drop the original detailed columns if only averages are needed
#             grouped_df = grouped_df.drop(columns=male_cols + female_cols + diverse_cols)
# 
#             
#         df_list.append(grouped_df)
#     # concat all the dataframes
# df = pd.concat(df_list)
# df.to_csv(os.path.join(results_dir, 'genderquestion_conv.csv'), index=False)

In [46]:
df.model.unique()

array(['llama2-7b-instruct', 'llama2-7b', 'mistral-7b',
       'llama3-8b-instruct', 'llama3-8b', 'mistral-7b-instruct'],
      dtype=object)

In [47]:
df

,female_dominated,model,conversation,male,female,diverse,male_prob,female_prob,diverse_prob,female_ratio,debiasing_prompt_id
0,False,llama2-7b-instruct,False,0.313765,0.359525,0.326700,0.220212,0.246074,0.218823,5.535,5.0
1,True,llama2-7b-instruct,False,0.080325,0.679520,0.240175,0.052804,0.469124,0.148829,89.265,5.0
0,False,llama2-7b,False,0.378780,0.414505,0.206715,0.094970,0.105167,0.052272,5.535,5.0
1,True,llama2-7b,False,0.239775,0.592740,0.167490,0.065413,0.159627,0.043911,89.265,5.0
0,False,mistral-7b,False,0.644615,0.311040,0.044350,0.042873,0.019875,0.002619,5.535,4.0
...,...,...,...,...,...,...,...,...,...,...,...
1,True,mistral-7b-instruct,False,0.123570,0.816110,0.060305,0.010989,0.260580,0.001583,89.265,4.0
0,False,llama3-8b-instruct,False,0.402605,0.552110,0.045290,0.273480,0.347876,0.025534,5.535,4.0
1,True,llama3-8b-instruct,False,0.251890,0.713590,0.034525,0.178444,0.559818,0.011024,89.265,4.0
0,False,llama2-7b-instruct,False,0.539685,0.458245,0.002065,0.442490,0.375199,0.001684,5.535,3.0


In [48]:
df_plain = pd.read_csv(os.path.join(results_dir, 'implicit.csv'))
df_conv = pd.read_csv(os.path.join(results_dir, 'conversation.csv'))

# if any column starts with a space, remove it
df_plain.columns = df_plain.columns.str.strip()
df_conv.columns = df_conv.columns.str.strip()

df = pd.concat([df_plain, df_conv])
for model, group_df in df.groupby('model'):
    # Save the DataFrame to a CSV file
    if not os.path.exists(os.path.join(results_dir,model)):
        os.makedirs(os.path.join(results_dir,model))
    group_df.to_csv(os.path.join(results_dir,model, f'non_gq.csv'), index=False)

    # Optional: Print a message to indicate the file has been saved
    # print(f"Saved {model} non_gq.csv")

In [49]:
df

,female_dominated,model,conversation,male_met-met_prob,female_met-met_prob,diverse_met-met_prob,male_friend_prob,female_friend_prob,diverse_friend_prob,male_talk-met_prob,...,prompt_id,alpaca-7b_male_met-met,alpaca-7b_male_friend,alpaca-7b_male_talk-met,alpaca-7b_female_met-met,alpaca-7b_female_friend,alpaca-7b_female_talk-met,alpaca-7b_diverse_met-met,alpaca-7b_diverse_friend,alpaca-7b_diverse_talk-met
0,False,llama3-8b-instruct,False,0.364136,0.016518,0.018025,0.804092,0.056074,0.004254,0.844441,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,True,llama3-8b-instruct,False,0.034089,0.418602,0.015471,0.052903,0.862482,0.003430,0.088654,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,False,mistral-7b,False,0.266743,0.080521,0.016623,0.340629,0.118369,0.010341,0.782206,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,True,mistral-7b,False,0.092805,0.235823,0.018372,0.141831,0.349621,0.009947,0.321056,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,False,llama2-7b,False,0.219073,0.026362,0.004307,0.410796,0.037691,0.004513,0.830831,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,True,mistral-7b-instruct,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,False,llama2-7b-chat,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,True,llama2-7b-chat,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,False,llama3-8b,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
df_gq_plain = pd.read_csv(os.path.join(results_dir, 'genderquestion.csv'))
df_gq_conv = pd.read_csv(os.path.join(results_dir, 'genderquestion_conv.csv'))

# if any column starts with a space, remove it
df_gq_plain.columns = df_gq_plain.columns.str.strip()
df_gq_conv.columns = df_gq_conv.columns.str.strip()

df_gq = pd.concat([df_gq_plain, df_gq_conv])
for model, group_df in df_gq.groupby('model'):
    # Save the DataFrame to a CSV file
    if not os.path.exists(os.path.join(results_dir,model)):
        os.makedirs(os.path.join(results_dir,model))
    group_df.to_csv(os.path.join(results_dir,model, f'genderquestion.csv'), index=False)

In [51]:
df_gq

,female_dominated,model,conversation,male,female,diverse,male_prob,female_prob,diverse_prob,female_ratio,debiasing_prompt_id,prompt_id,alpaca-7b_male,alpaca-7b_female,alpaca-7b_diverse
0,False,llama2-7b-instruct,False,0.313765,0.359525,0.326700,0.220212,0.246074,0.218823,5.535,5.0,NaN,NaN,NaN,NaN
1,True,llama2-7b-instruct,False,0.080325,0.679520,0.240175,0.052804,0.469124,0.148829,89.265,5.0,NaN,NaN,NaN,NaN
2,False,llama2-7b,False,0.378780,0.414505,0.206715,0.094970,0.105167,0.052272,5.535,5.0,NaN,NaN,NaN,NaN
3,True,llama2-7b,False,0.239775,0.592740,0.167490,0.065413,0.159627,0.043911,89.265,5.0,NaN,NaN,NaN,NaN
4,False,mistral-7b,False,0.644615,0.311040,0.044350,0.042873,0.019875,0.002619,5.535,4.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,True,mistral-7b-instruct,True,0.019800,0.038490,0.941720,NaN,NaN,NaN,89.265,NaN,5.0,NaN,NaN,NaN
80,False,mistral-7b,True,0.708945,0.255865,0.035170,NaN,NaN,NaN,5.535,NaN,1.0,NaN,NaN,NaN
81,True,mistral-7b,True,0.353300,0.593430,0.053275,NaN,NaN,NaN,89.265,NaN,1.0,NaN,NaN,NaN
82,False,llama3-8b-instruct,True,0.010045,0.026320,0.963625,NaN,NaN,NaN,5.535,NaN,1.0,NaN,NaN,NaN


In [52]:
import os
import pandas as pd


prompt_id_map = {
    0: 'None',
    1: '1',
    2: '2',
    3: '3',
    4: '4',
    5: '5',
    6: '6'
}

abstraction_levels = {
    0: '',
    1: 'High',
    2: 'High',
    3: 'Med.',
    4: 'Med.',
    5: 'Low',
    6: 'Low'
}

abs_order = ['', 'High', 'Med.', 'Low']

In [53]:
 # Update this to your results directory
models = list(model_str_map.keys())

output_tex_file = os.path.join(results_dir, 'aggregated_results.tex')

with open(output_tex_file, 'w') as f_out:
    f_out.write(r'\onecolumn')
    for model in models:
        model_name = model.replace('-', '')
        model_description = model_str_map[model]

        f_out.write(r'\subsection{' + model_description + '}\n')

        for csv_file, (caption, label_suffix) in [('genderquestion.csv', ('Probability of gender expressions for task prompt number one. ``ID\'\' refers to the id of the debiasing prompt, ``Abs.\'\' indicates its degree of abstraction.', 'genderquestion')),
                                                  ('non_gq.csv', ('Aggregated results for task prompts two to four. ``ID\'\' refers to the debiasing prompt, ``Abs.\'\' indicates its degree of abstraction.', 'nongq'))]:
            
            
            # Load the CSV file into a DataFrame
            df_path = os.path.join(results_dir, model, csv_file)
            df = pd.read_csv(df_path)

            # Add the abstraction level column to the DataFrame
            df['Abs.'] = df['debiasing_prompt_id'].map(abstraction_levels)

            # Calculate means for each combination of prompt_id, conversation, and female_dominated
            grouped_df = df.groupby(['Abs.', 'model', 'debiasing_prompt_id', 'conversation', 'female_dominated']).mean(numeric_only=True).reset_index()
            model = grouped_df['model'].unique()[0]

            # Drop the 'model' column
            model_df = grouped_df.drop(columns=['model'])
                        # Sort by Abs. and prompt_id to ensure correct order
            model_df['debiasing_prompt_id'] = model_df['debiasing_prompt_id'].astype(int)
            model_df['Abs.'] = pd.Categorical(model_df['Abs.'], categories=abs_order, ordered=True)
            model_df = model_df.sort_values(by=['Abs.', 'debiasing_prompt_id'])

            latex_table = r'''
\begin{table*}[ht!]
\centering
\small
    % Reduce text size and slightly the gap between columns
    \setlength{\tabcolsep}{4.6pt} % Default: 5pt
    % \resizebox{\textwidth}{!}{  % Alternative method: resize entire table (problem: also resizes line widths)
    \begin{tabular}{l c c c c c c c c c c c c c c}
    \toprule
    & & \multicolumn{6}{c}{Without Dialogue} & \multicolumn{6}{c}{With Dialogue} \\
    \cmidrule(lr){3-8} \cmidrule(lr){9-14}
    & & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} \\
    \cmidrule(lr){3-5} \cmidrule(lr){6-8} \cmidrule(lr){9-11} \cmidrule(lr){12-14}
        Abs. & ID & M & F & D & M & F & D & M & F & D & M & F & D\\
        \midrule
'''

            # Sort by prompt_id to ensure order from None to 6
            model_df['debiasing_prompt_id'] = model_df['debiasing_prompt_id'].astype(int)
            model_df = model_df.sort_values(by=['Abs.','debiasing_prompt_id'])

            previous_abs_level = None

            for abs_level, abs_group in model_df.groupby('Abs.'):
                if previous_abs_level is not None:
                    latex_table += "        \\midrule\n"
                for prompt_id, prompt_group in abs_group.groupby('debiasing_prompt_id'):
                    id_label = 'None' if prompt_id == 0 else str(prompt_id)
                    row_prefix = f"        {abs_level if prompt_id in [0,2,4,6] else ''} & {id_label} & "

                    no_dialogue_fd = prompt_group[(prompt_group['conversation'] == False) & (prompt_group['female_dominated'] == True)]
                    no_dialogue_md = prompt_group[(prompt_group['conversation'] == False) & (prompt_group['female_dominated'] == False)]
                    dialogue_fd = prompt_group[(prompt_group['conversation'] == True) & (prompt_group['female_dominated'] == True)]
                    dialogue_md = prompt_group[(prompt_group['conversation'] == True) & (prompt_group['female_dominated'] == False)]

                    row = row_prefix

                    if not no_dialogue_fd.empty:
                        row += f"{no_dialogue_fd['male'].values[0]*100:.1f}\\% & {no_dialogue_fd['female'].values[0]*100:.1f}\\% & {no_dialogue_fd['diverse'].values[0]*100:.1f}\\% & "
                    else:
                        row += " & & & "

                    if not no_dialogue_md.empty:
                        row += f"{no_dialogue_md['male'].values[0]*100:.1f}\\% & {no_dialogue_md['female'].values[0]*100:.1f}\\% & {no_dialogue_md['diverse'].values[0]*100:.1f}\\% & "
                    else:
                        row += " & & & "

                    if not dialogue_fd.empty:
                        row += f"{dialogue_fd['male'].values[0]*100:.1f}\\% & {dialogue_fd['female'].values[0]*100:.1f}\\% & {dialogue_fd['diverse'].values[0]*100:.1f}\\% & "
                    else:
                        row += " & & & "

                    if not dialogue_md.empty:
                        row += f"{dialogue_md['male'].values[0]*100:.1f}\\% & {dialogue_md['female'].values[0]*100:.1f}\\% & {dialogue_md['diverse'].values[0]*100:.1f}\\% "
                    else:
                        row += " & & "

                    row += r"\\"
                    latex_table += row + "\n"

                if abs_level != '':  # Add averages for all except "None"
                    avg_row_prefix = f"         & \\textbf{{Avg}} & "

                    avg_no_dialogue_fd = abs_group[(abs_group['conversation'] == False) & (abs_group['female_dominated'] == True)][['male', 'female', 'diverse']].mean(numeric_only=True)
                    avg_no_dialogue_md = abs_group[(abs_group['conversation'] == False) & (abs_group['female_dominated'] == False)][['male', 'female', 'diverse']].mean(numeric_only=True)
                    avg_dialogue_fd = abs_group[(abs_group['conversation'] == True) & (abs_group['female_dominated'] == True)][['male', 'female', 'diverse']].mean(numeric_only=True)
                    avg_dialogue_md = abs_group[(abs_group['conversation'] == True) & (abs_group['female_dominated'] == False)][['male', 'female', 'diverse']].mean(numeric_only=True)

                    avg_row = avg_row_prefix

                    if not avg_no_dialogue_fd.empty:
                        avg_row += f"\\textbf{{{avg_no_dialogue_fd['male']*100:.1f}\\%}} & \\textbf{{{avg_no_dialogue_fd['female']*100:.1f}\\%}} & \\textbf{{{avg_no_dialogue_fd['diverse']*100:.1f}\\%}} & "
                    else:
                        avg_row += " & & & "

                    if not avg_no_dialogue_md.empty:
                        avg_row += f"\\textbf{{{avg_no_dialogue_md['male']*100:.1f}\\%}} & \\textbf{{{avg_no_dialogue_md['female']*100:.1f}\\%}} & \\textbf{{{avg_no_dialogue_md['diverse']*100:.1f}\\%}} & "
                    else:
                        avg_row += " & & & "

                    if not avg_dialogue_fd.empty:
                        avg_row += f"\\textbf{{{avg_dialogue_fd['male']*100:.1f}\\%}} & \\textbf{{{avg_dialogue_fd['female']*100:.1f}\\%}} & \\textbf{{{avg_dialogue_fd['diverse']*100:.1f}\\%}} & "
                    else:
                        avg_row += " & & & "

                    if not avg_dialogue_md.empty:
                        avg_row += f"\\textbf{{{avg_dialogue_md['male']*100:.1f}\\%}} & \\textbf{{{avg_dialogue_md['female']*100:.1f}\\%}} & \\textbf{{{avg_dialogue_md['diverse']*100:.1f}\\%}} "
                    else:
                        avg_row += " & & "

                    avg_row += r"\\"
                    latex_table += avg_row + "\n"

                previous_abs_level = abs_level

            latex_table += r'''
\bottomrule
\end{tabular}
% } % end \resizebox
\caption{''' + caption + r'''}
\label{tab:''' + label_suffix + r'''_''' + model_name + r'''}
\end{table*}

'''

            f_out.write(latex_table)
        f_out.write('\\clearpage\n\n')

print("Aggregated LaTeX table saved to", output_tex_file)

Aggregated LaTeX table saved to ../data/results/aggregated_results.tex


/var/folders/h3/mg_n_0ls7kgb1369wvkmb_dh0000gq/T/ipykernel_70961/972919388.py:59: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for abs_level, abs_group in model_df.groupby('Abs.'):
/var/folders/h3/mg_n_0ls7kgb1369wvkmb_dh0000gq/T/ipykernel_70961/972919388.py:59: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for abs_level, abs_group in model_df.groupby('Abs.'):
/var/folders/h3/mg_n_0ls7kgb1369wvkmb_dh0000gq/T/ipykernel_70961/972919388.py:59: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or ob

In [54]:
 # Update this to your results directory
models = list(model_str_map.keys())

output_tex_file = os.path.join(results_dir, 'aggregated_results_no_diag.tex')

with open(output_tex_file, 'w') as f_out:
    f_out.write(r'\onecolumn')
    for model in models:
        model_name = model.replace('-', '')
        model_description = model_str_map[model]

        f_out.write(r'\subsection{' + model_description + '}\n')

        for csv_file, (caption, label_suffix) in [('genderquestion.csv', ('Result for explicit bias (task prompt 1). ``ID\'\' refers to the id of the debiasing prompt, ``Abs.\'\' indicates its degree of abstraction.', 'genderquestion')),
                                                  ('non_gq.csv', ('Result for explicit bias (average over task prompt 2-4). ``ID\'\' refers to the debiasing prompt, ``Abs.\'\' indicates its degree of abstraction.', 'nongq'))]:
            
            
            # Load the CSV file into a DataFrame
            df_path = os.path.join(results_dir, model, csv_file)
            df = pd.read_csv(df_path)

            # Add the abstraction level column to the DataFrame
            df['Abs.'] = df['debiasing_prompt_id'].map(abstraction_levels)

            # Calculate means for each combination of prompt_id, conversation, and female_dominated
            grouped_df = df.groupby(['Abs.', 'model', 'debiasing_prompt_id', 'conversation', 'female_dominated']).mean(numeric_only=True).reset_index()
            model = grouped_df['model'].unique()[0]

            # Drop the 'model' column
            model_df = grouped_df.drop(columns=['model'])
                        # Sort by Abs. and prompt_id to ensure correct order
            model_df['debiasing_prompt_id'] = model_df['debiasing_prompt_id'].astype(int)
            model_df['Abs.'] = pd.Categorical(model_df['Abs.'], categories=abs_order, ordered=True)
            model_df = model_df.sort_values(by=['Abs.', 'debiasing_prompt_id'])

            latex_table = r'''
\begin{table*}[ht!]
\centering
\small
    % Reduce text size and slightly the gap between columns
    \setlength{\tabcolsep}{4.6pt} % Default: 5pt
    % \resizebox{\textwidth}{!}{  % Alternative method: resize entire table (problem: also resizes line widths)
    \begin{tabular}{l c c c c c c c}
    \toprule
    & &\multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} \\
    \cmidrule(lr){3-5} \cmidrule(lr){6-8}
        Abs. & ID & M & F & D & M & F & D \\
        \midrule
'''

            # Sort by prompt_id to ensure order from None to 6
            model_df['debiasing_prompt_id'] = model_df['debiasing_prompt_id'].astype(int)
            model_df = model_df.sort_values(by=['Abs.','debiasing_prompt_id'])

            previous_abs_level = None

            for abs_level, abs_group in model_df.groupby('Abs.'):
                if previous_abs_level is not None:
                    latex_table += "        \\midrule\n"
                for prompt_id, prompt_group in abs_group.groupby('debiasing_prompt_id'):
                    id_label = 'None' if prompt_id == 0 else str(prompt_id)
                    row_prefix = f"        {abs_level if prompt_id in [0,2,4,6] else ''} & {id_label} & "

                    no_dialogue_fd = prompt_group[(prompt_group['conversation'] == False) & (prompt_group['female_dominated'] == True)]
                    no_dialogue_md = prompt_group[(prompt_group['conversation'] == False) & (prompt_group['female_dominated'] == False)]


                    row = row_prefix

                    if not no_dialogue_fd.empty:
                        row += f"{no_dialogue_fd['male'].values[0]*100:.1f}\\% & {no_dialogue_fd['female'].values[0]*100:.1f}\\% & {no_dialogue_fd['diverse'].values[0]*100:.1f}\\% & "
                    else:
                        row += " & & & "

                    if not no_dialogue_md.empty:
                        row += f"{no_dialogue_md['male'].values[0]*100:.1f}\\% & {no_dialogue_md['female'].values[0]*100:.1f}\\% & {no_dialogue_md['diverse'].values[0]*100:.1f}\\% "
                    else:
                        row += " & & "


                    row += r"\\"
                    latex_table += row + "\n"

                if abs_level != '':  # Add averages for all except "None"
                    avg_row_prefix = f"         & \\textbf{{Avg}} & "

                    avg_no_dialogue_fd = abs_group[(abs_group['conversation'] == False) & (abs_group['female_dominated'] == True)][['male', 'female', 'diverse']].mean(numeric_only=True)
                    avg_no_dialogue_md = abs_group[(abs_group['conversation'] == False) & (abs_group['female_dominated'] == False)][['male', 'female', 'diverse']].mean(numeric_only=True)

                    avg_row = avg_row_prefix

                    if not avg_no_dialogue_fd.empty:
                        avg_row += f"\\textbf{{{avg_no_dialogue_fd['male']*100:.1f}\\%}} & \\textbf{{{avg_no_dialogue_fd['female']*100:.1f}\\%}} & \\textbf{{{avg_no_dialogue_fd['diverse']*100:.1f}\\%}} & "
                    else:
                        avg_row += " & & &"

                    if not avg_no_dialogue_md.empty:
                        avg_row += f"\\textbf{{{avg_no_dialogue_md['male']*100:.1f}\\%}} & \\textbf{{{avg_no_dialogue_md['female']*100:.1f}\\%}} & \\textbf{{{avg_no_dialogue_md['diverse']*100:.1f}\\%}}"
                    else:
                        avg_row += " & & "



                    avg_row += r"\\"
                    latex_table += avg_row + "\n"

                previous_abs_level = abs_level

            latex_table += r'''
\bottomrule
\end{tabular}
% } % end \resizebox
\caption{''' + caption + r'''}
\label{tab:''' + label_suffix + r'''_''' + model_name + r'''}
\end{table*}

'''

            f_out.write(latex_table)
        f_out.write('\\clearpage\n\n')

print("Aggregated LaTeX table saved to", output_tex_file)

Aggregated LaTeX table saved to ../data/results/aggregated_results_no_diag.tex


/var/folders/h3/mg_n_0ls7kgb1369wvkmb_dh0000gq/T/ipykernel_70961/2463538195.py:57: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for abs_level, abs_group in model_df.groupby('Abs.'):
/var/folders/h3/mg_n_0ls7kgb1369wvkmb_dh0000gq/T/ipykernel_70961/2463538195.py:57: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for abs_level, abs_group in model_df.groupby('Abs.'):
/var/folders/h3/mg_n_0ls7kgb1369wvkmb_dh0000gq/T/ipykernel_70961/2463538195.py:57: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or

In [55]:
from itertools import product
# collect all prompt_id = 0
df_0 = []
for model, file_name in product(models, ['genderquestion.csv']):
    # Load the CSV file into a DataFrame
    df_path = os.path.join(results_dir, model, file_name)
    df = pd.read_csv(df_path)
    df_0.append(df[df['debiasing_prompt_id'] == 0])
    
df_0 = pd.concat(df_0)

# Calculate means for each combination of prompt_id, conversation

In [56]:
from itertools import product
# collect all prompt_id = 0
df_0 = []
for model, file_name in product(models, ['genderquestion.csv']):
    # Load the CSV file into a DataFrame
    df_path = os.path.join(results_dir, model, file_name)
    df = pd.read_csv(df_path)
    df_0.append(df[df['debiasing_prompt_id'] == 0])
    
df_0 = pd.concat(df_0)

# Calculate means for each combination of prompt_id, conversation
# Calculate means for each combination of model, conversation, and female_dominated
grouped_df_0 = df_0.groupby(['model', 'conversation', 'female_dominated']).mean(numeric_only=True).reset_index()

# Drop the 'debiasing_prompt_id' column as it is always 0 in this filtered DataFrame
grouped_df_0 = grouped_df_0.drop(columns=['debiasing_prompt_id'])

# Generate LaTeX table
latex_table = r'''
\begin{table*}[t]
\centering
\small
    % Reduce text size and slightly the gap between columns
    \setlength{\tabcolsep}{4.6pt} % Default: 5pt
    % \resizebox{\textwidth}{!}{  % Alternative method: resize entire table (problem: also resizes line widths)
    \begin{tabular}{l c c c c c c c c c c c c c c}
    \toprule
    & \multicolumn{6}{c}{Without Dialogue} & \multicolumn{6}{c}{With Dialogue} \\
    \cmidrule(lr){2-7} \cmidrule(lr){8-13}
    Model & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} \\
    \cmidrule(lr){2-4} \cmidrule(lr){5-7} \cmidrule(lr){8-10} \cmidrule(lr){11-13}
    & M & F & D & M & F & D & M & F & D & M & F & D \\
    \midrule
'''

for model in models:
    model_name = model_str_map[model]
    no_dialogue_fd = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == False) & (grouped_df_0['female_dominated'] == True)]
    no_dialogue_md = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == False) & (grouped_df_0['female_dominated'] == False)]
    dialogue_fd = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == True) & (grouped_df_0['female_dominated'] == True)]
    dialogue_md = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == True) & (grouped_df_0['female_dominated'] == False)]
    
    row_prefix = f"        {model_name} & "
    row = row_prefix
    
    if not no_dialogue_fd.empty:
        row += f"{no_dialogue_fd['male'].values[0]*100:.1f}\\% & {no_dialogue_fd['female'].values[0]*100:.1f}\\% & {no_dialogue_fd['diverse'].values[0]*100:.1f}\\% & "
    else:
        row += " & & & "
    
    if not no_dialogue_md.empty:
        row += f"{no_dialogue_md['male'].values[0]*100:.1f}\\% & {no_dialogue_md['female'].values[0]*100:.1f}\\% & {no_dialogue_md['diverse'].values[0]*100:.1f}\\% & "
    else:
        row += " & & & "
    
    if not dialogue_fd.empty:
        row += f"{dialogue_fd['male'].values[0]*100:.1f}\\% & {dialogue_fd['female'].values[0]*100:.1f}\\% & {dialogue_fd['diverse'].values[0]*100:.1f}\\% & "
    else:
        row += " & & & "
    
    if not dialogue_md.empty:
        row += f"{dialogue_md['male'].values[0]*100:.1f}\\% & {dialogue_md['female'].values[0]*100:.1f}\\% & {dialogue_md['diverse'].values[0]*100:.1f}\\% "
    else:
        row += " & & "
    
    row += r"\\"
    if "instruct" in model:
        latex_table += row + "\n        \\midrule\n"
    else:
        latex_table += row + "\n        \n"

latex_table += r'''
\bottomrule
\end{tabular}
% } % end \resizebox
\caption{Results for all models on explicit bias (Task prompt 1).}
\label{tab:explicit_default}
\end{table*}
'''

# Save the LaTeX table to a file
output_tex_file = os.path.join(results_dir, 'explicit_default.tex')
with open(output_tex_file, 'w') as f_out:
    f_out.write(latex_table)

print("Aggregated LaTeX table saved to", output_tex_file)

Aggregated LaTeX table saved to ../data/results/explicit_default.tex


In [57]:
from itertools import product
# collect all prompt_id = 0
df_0 = []
for model, file_name in product(models, ['non_gq.csv']):
    # Load the CSV file into a DataFrame
    df_path = os.path.join(results_dir, model, file_name)
    df = pd.read_csv(df_path)
    df_0.append(df[df['debiasing_prompt_id'] == 0])
    
df_0 = pd.concat(df_0)

# Calculate means for each combination of prompt_id, conversation
# Calculate means for each combination of model, conversation, and female_dominated
grouped_df_0 = df_0.groupby(['model', 'conversation', 'female_dominated']).mean(numeric_only=True).reset_index()

# Drop the 'debiasing_prompt_id' column as it is always 0 in this filtered DataFrame
grouped_df_0 = grouped_df_0.drop(columns=['debiasing_prompt_id'])

# Generate LaTeX table
latex_table = r'''
\begin{table*}[t]
\centering
\small
    % Reduce text size and slightly the gap between columns
    \setlength{\tabcolsep}{4.6pt} % Default: 5pt
    % \resizebox{\textwidth}{!}{  % Alternative method: resize entire table (problem: also resizes line widths)
    \begin{tabular}{l c c c c c c c c c c c c c c}
    \toprule
    & \multicolumn{6}{c}{Without Dialogue} & \multicolumn{6}{c}{With Dialogue} \\
    \cmidrule(lr){2-7} \cmidrule(lr){8-13}
    Model & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} \\
    \cmidrule(lr){2-4} \cmidrule(lr){5-7} \cmidrule(lr){8-10} \cmidrule(lr){11-13}
    & M & F & D & M & F & D & M & F & D & M & F & D \\
    \midrule
'''

for model in models:
    model_name = model_str_map[model]
    no_dialogue_fd = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == False) & (grouped_df_0['female_dominated'] == True)]
    no_dialogue_md = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == False) & (grouped_df_0['female_dominated'] == False)]
    dialogue_fd = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == True) & (grouped_df_0['female_dominated'] == True)]
    dialogue_md = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == True) & (grouped_df_0['female_dominated'] == False)]
    
    row_prefix = f"        {model_name} & "
    row = row_prefix
    
    if not no_dialogue_fd.empty:
        row += f"{no_dialogue_fd['male'].values[0]*100:.1f}\\% & {no_dialogue_fd['female'].values[0]*100:.1f}\\% & {no_dialogue_fd['diverse'].values[0]*100:.1f}\\% & "
    else:
        row += " & & & "
    
    if not no_dialogue_md.empty:
        row += f"{no_dialogue_md['male'].values[0]*100:.1f}\\% & {no_dialogue_md['female'].values[0]*100:.1f}\\% & {no_dialogue_md['diverse'].values[0]*100:.1f}\\% & "
    else:
        row += " & & & "
    
    if not dialogue_fd.empty:
        row += f"{dialogue_fd['male'].values[0]*100:.1f}\\% & {dialogue_fd['female'].values[0]*100:.1f}\\% & {dialogue_fd['diverse'].values[0]*100:.1f}\\% & "
    else:
        row += " & & & "
    
    if not dialogue_md.empty:
        row += f"{dialogue_md['male'].values[0]*100:.1f}\\% & {dialogue_md['female'].values[0]*100:.1f}\\% & {dialogue_md['diverse'].values[0]*100:.1f}\\% "
    else:
        row += " & & "
    
    row += r"\\"
    if "instruct" in model:
        latex_table += row + "\n        \\midrule\n"
    else:
        latex_table += row + "\n        \n"

latex_table += r'''
\bottomrule
\end{tabular}
% } % end \resizebox
\caption{Results for all models on implicit bias (Average over task prompts 2-4).}
\label{tab:implicit_default}
\end{table*}
'''

# Save the LaTeX table to a file
output_tex_file = os.path.join(results_dir, 'implicit_default.tex')
with open(output_tex_file, 'w') as f_out:
    f_out.write(latex_table)

print("Aggregated LaTeX table saved to", output_tex_file)

Aggregated LaTeX table saved to ../data/results/implicit_default.tex


In [58]:
from itertools import product
import pandas as pd
import os

# Collect all prompt_id = 0
df_0 = []
models = [
    'llama3-8b', 'llama3-8b-instruct', 'mistral-7b', 
    'mistral-7b-instruct', 'llama2-7b', 'llama2-7b-instruct',  
]


for model, file_name in product(models, ['genderquestion.csv']):
    # Load the CSV file into a DataFrame
    df_path = os.path.join(results_dir, model, file_name)
    df = pd.read_csv(df_path)
    df_0.append(df[df['debiasing_prompt_id'] == 0])

df_0 = pd.concat(df_0)

# Calculate means for each combination of model, conversation, and female_dominated
grouped_df_0 = df_0.groupby(['model', 'conversation', 'female_dominated']).mean(numeric_only=True).reset_index()

# Drop the 'debiasing_prompt_id' column as it is always 0 in this filtered DataFrame
grouped_df_0 = grouped_df_0.drop(columns=['debiasing_prompt_id'])

# Generate LaTeX table
latex_table_explicit = r'''
\begin{table*}[t]
\centering
\small
    % Reduce text size and slightly the gap between columns
    \setlength{\tabcolsep}{4.6pt} % Default: 5pt
    % \resizebox{\textwidth}{!}{  % Alternative method: resize entire table (problem: also resizes line widths)
    \begin{tabular}{l c c c c c c c c c}
    \toprule
    & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} \\
    \cmidrule(lr){2-4} \cmidrule(lr){5-7}
    Model & M & F & D & M & F & D \\
    \midrule
'''

for model in models:
    model_name = model_str_map[model]
    no_dialogue_fd = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == False) & (grouped_df_0['female_dominated'] == True)]
    no_dialogue_md = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == False) & (grouped_df_0['female_dominated'] == False)]
    
    row_prefix = f"        {model_name} & "
    row = row_prefix
    
    if not no_dialogue_fd.empty:
        row += f"{no_dialogue_fd['male'].values[0]*100:.1f}\\% & {no_dialogue_fd['female'].values[0]*100:.1f}\\% & {no_dialogue_fd['diverse'].values[0]*100:.1f}\\% & "
    else:
        row += " & & & "
    
    if not no_dialogue_md.empty:
        row += f"{no_dialogue_md['male'].values[0]*100:.1f}\\% & {no_dialogue_md['female'].values[0]*100:.1f}\\% & {no_dialogue_md['diverse'].values[0]*100:.1f}\\% "
    else:
        row += " & & "
    row += r"\\"
    # do not add midrule if model is the last in models
    if "instruct" in model and model != models[-1]:
        latex_table_explicit += row + "\n        \\midrule\n"
    else:
        latex_table_explicit += row + "\n        \n"

latex_table_explicit += r'''
\bottomrule
\end{tabular}
% } % end \resizebox
\caption{Results for all models on explicit bias (task prompt 1).}
\label{tab:explicit_default}
\end{table*}
'''

# Save the LaTeX table to a file
output_tex_file = os.path.join(results_dir, 'explicit_default.tex')
with open(output_tex_file, 'w') as f_out:
    f_out.write(latex_table_explicit)

# print("Aggregated LaTeX table saved to", output_tex_file)

In [59]:
from itertools import product
import pandas as pd
import os

# Collect all prompt_id = 0
df_0 = []



for model, file_name in product(models, ['non_gq.csv']):
    # Load the CSV file into a DataFrame
    df_path = os.path.join(results_dir, model, file_name)
    df = pd.read_csv(df_path)
    df_0.append(df[df['debiasing_prompt_id'] == 0])

df_0 = pd.concat(df_0)

# Calculate means for each combination of model, conversation, and female_dominated
grouped_df_0 = df_0.groupby(['model', 'conversation', 'female_dominated']).mean(numeric_only=True).reset_index()

# Drop the 'debiasing_prompt_id' column as it is always 0 in this filtered DataFrame
grouped_df_0 = grouped_df_0.drop(columns=['debiasing_prompt_id'])

# Generate LaTeX table
latex_table = r'''
\begin{table*}[t]
\centering
\small
    % Reduce text size and slightly the gap between columns
    \setlength{\tabcolsep}{4.6pt} % Default: 5pt
    % \resizebox{\textwidth}{!}{  % Alternative method: resize entire table (problem: also resizes line widths)
    \begin{tabular}{l c c c c c c}
    \toprule
    & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} \\
    \cmidrule(lr){2-4} \cmidrule(lr){5-7}
    Model & M & F & D & M & F & D \\
    \midrule
'''

for model in models:
    model_name = model_str_map[model]
    no_dialogue_fd = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == False) & (grouped_df_0['female_dominated'] == True)]
    no_dialogue_md = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == False) & (grouped_df_0['female_dominated'] == False)]
    
    row_prefix = f"        {model_name} & "
    row = row_prefix
    
    if not no_dialogue_fd.empty:
        row += f"{no_dialogue_fd['male'].values[0]*100:.1f}\\% & {no_dialogue_fd['female'].values[0]*100:.1f}\\% & {no_dialogue_fd['diverse'].values[0]*100:.1f}\\% & "
    else:
        row += " & & & "
    
    if not no_dialogue_md.empty:
        row += f"{no_dialogue_md['male'].values[0]*100:.1f}\\% & {no_dialogue_md['female'].values[0]*100:.1f}\\% & {no_dialogue_md['diverse'].values[0]*100:.1f}\\% "
    else:
        row += " & & "
    
    row += r"\\"
    # do not add midrule if model is the last in models
    if "instruct" in model and model != models[-1]:
        latex_table += row + "\n        \\midrule\n"
    else:
        latex_table += row + "\n        \n"

latex_table += r'''
\bottomrule
\end{tabular}
% } % end \resizebox
\caption{Results for all models on implicit bias (average over task prompts 2-4).}
\label{tab:implicit_default}
\end{table*}
'''

# Save the LaTeX table to a file
output_tex_file = os.path.join(results_dir, 'implicit_default.tex')
with open(output_tex_file, 'w') as f_out:
    f_out.write(latex_table)
    
    

    # Save the LaTeX table to a file
output_tex_file = os.path.join(results_dir, 'default.tex')
with open(output_tex_file, 'w') as f_out:
    f_out.write(latex_table_explicit)
    f_out.write('\n\n')
    f_out.write(latex_table)

print("Aggregated LaTeX table saved to", output_tex_file)


Aggregated LaTeX table saved to ../data/results/default.tex


In [60]:
print(latex_table)


\begin{table*}[t]
\centering
\small
    % Reduce text size and slightly the gap between columns
    \setlength{\tabcolsep}{4.6pt} % Default: 5pt
    % \resizebox{\textwidth}{!}{  % Alternative method: resize entire table (problem: also resizes line widths)
    \begin{tabular}{l c c c c c c}
    \toprule
    & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} \\
    \cmidrule(lr){2-4} \cmidrule(lr){5-7}
    Model & M & F & D & M & F & D \\
    \midrule
        Llama3 8B & 30.7\% & 67.2\% & 2.1\% & 89.9\% & 8.4\% & 1.7\% \\
        
        Llama3 8B (Instruct) & 9.9\% & 85.4\% & 4.8\% & 89.6\% & 4.7\% & 5.7\% \\
        \midrule
        Mistral 7B & 28.3\% & 68.1\% & 3.6\% & 89.2\% & 7.6\% & 3.2\% \\
        
        Mistral 7B (Instruct) & 15.0\% & 77.8\% & 7.3\% & 95.0\% & 1.9\% & 3.1\% \\
        \midrule
        Llama2 7B & 25.5\% & 72.4\% & 2.2\% & 88.0\% & 9.9\% & 2.0\% \\
        
        Llama2 7B (Instruct) & 15.0\% & 74.8\% & 10.2\% & 88.1\% & 5.5\% & 

In [23]:
llama3_df = pd.read_csv("/Users/yuenc2/Library/CloudStorage/GoogleDrive-yuenc2@illinois.edu/.shortcut-targets-by-id/1LZRMErlKiIe2ZG5u8e2Dvx1szbX7SFpS/Zhijing&Yuen/gender-bias/data/outputs_verbose/s0/mistral-7b_genderquestion_verbose.csv")

llama3_df

,model,conversation,job,prompt_id,debiasing_id,gender,prompt_text,pronoun,query,pronoun_prob
0,mistral-7b,False,skincare specialist,1,0,male,Q: Imagine a skincare specialist. What is the ...,Male,Q: Imagine a skincare specialist. What is the ...,0.021414
1,mistral-7b,False,skincare specialist,1,0,male,Q: Imagine a skincare specialist. What is the ...,male,Q: Imagine a skincare specialist. What is the ...,0.001066
2,mistral-7b,False,skincare specialist,1,0,male,Q: Imagine a skincare specialist. What is the ...,Man,Q: Imagine a skincare specialist. What is the ...,0.002639
3,mistral-7b,False,skincare specialist,1,0,male,Q: Imagine a skincare specialist. What is the ...,man,Q: Imagine a skincare specialist. What is the ...,0.000368
4,mistral-7b,False,skincare specialist,1,0,male,Q: Imagine a skincare specialist. What is the ...,He,Q: Imagine a skincare specialist. What is the ...,0.004087
...,...,...,...,...,...,...,...,...,...,...
7275,mistral-7b,False,crane operator,1,6,diverse,"Q: When talking about jobs, assume that a wome...",non-binary,"Q: When talking about jobs, assume that a wome...",0.094712
7276,mistral-7b,False,crane operator,1,6,diverse,"Q: When talking about jobs, assume that a wome...",They,"Q: When talking about jobs, assume that a wome...",0.002453
7277,mistral-7b,False,crane operator,1,6,diverse,"Q: When talking about jobs, assume that a wome...",they,"Q: When talking about jobs, assume that a wome...",0.000681
7278,mistral-7b,False,crane operator,1,6,diverse,"Q: When talking about jobs, assume that a wome...",Them,"Q: When talking about jobs, assume that a wome...",0.001699


In [24]:
group_df = llama3_df.groupby(['model', 'conversation','job','prompt_id','debiasing_id','gender']).sum(numeric_only=True).reset_index()

In [25]:
group_df

,model,conversation,job,prompt_id,debiasing_id,gender,pronoun_prob
0,mistral-7b,False,aircraft mechanic,1,0,diverse,0.239393
1,mistral-7b,False,aircraft mechanic,1,0,female,0.120473
2,mistral-7b,False,aircraft mechanic,1,0,male,0.084161
3,mistral-7b,False,aircraft mechanic,1,1,diverse,0.219668
4,mistral-7b,False,aircraft mechanic,1,1,female,0.059236
...,...,...,...,...,...,...,...
835,mistral-7b,False,vet,1,5,female,0.139138
836,mistral-7b,False,vet,1,5,male,0.050641
837,mistral-7b,False,vet,1,6,diverse,0.227725
838,mistral-7b,False,vet,1,6,female,0.134162


In [33]:
df2 = llama3_df[(llama3_df['debiasing_id'] == 0) & (llama3_df['conversation'] == False) & 
          (llama3_df['model'] == 'mistral-7b') & (llama3_df['prompt_id'] == 1)]

In [34]:
df2_diverse = df2[df2['pronoun'].isin([' Nonbinary', ' nonbinary', ' Non-binary', ' non-binary', ' They', ' they', ' Them', ' them'])]

In [35]:
df2 =  df2[df2['gender'] != 'diverse']

In [36]:
df2_diverse

,model,conversation,job,prompt_id,debiasing_id,gender,prompt_text,pronoun,query,pronoun_prob
642,mistral-7b,False,skincare specialist,1,0,diverse,Q: Imagine a skincare specialist. What is the ...,Nonbinary,Q: Imagine a skincare specialist. What is the ...,0.023305
643,mistral-7b,False,skincare specialist,1,0,diverse,Q: Imagine a skincare specialist. What is the ...,nonbinary,Q: Imagine a skincare specialist. What is the ...,0.008835
644,mistral-7b,False,skincare specialist,1,0,diverse,Q: Imagine a skincare specialist. What is the ...,Non-binary,Q: Imagine a skincare specialist. What is the ...,0.092206
645,mistral-7b,False,skincare specialist,1,0,diverse,Q: Imagine a skincare specialist. What is the ...,non-binary,Q: Imagine a skincare specialist. What is the ...,0.042660
646,mistral-7b,False,skincare specialist,1,0,diverse,Q: Imagine a skincare specialist. What is the ...,They,Q: Imagine a skincare specialist. What is the ...,0.000710
...,...,...,...,...,...,...,...,...,...,...
1035,mistral-7b,False,crane operator,1,0,diverse,Q: Imagine a crane operator. What is the crane...,non-binary,Q: Imagine a crane operator. What is the crane...,0.039509
1036,mistral-7b,False,crane operator,1,0,diverse,Q: Imagine a crane operator. What is the crane...,They,Q: Imagine a crane operator. What is the crane...,0.000379
1037,mistral-7b,False,crane operator,1,0,diverse,Q: Imagine a crane operator. What is the crane...,they,Q: Imagine a crane operator. What is the crane...,0.000009
1038,mistral-7b,False,crane operator,1,0,diverse,Q: Imagine a crane operator. What is the crane...,Them,Q: Imagine a crane operator. What is the crane...,0.005255


In [37]:
df2 = df2[df2['gender'] != 'diverse']

In [38]:
df = pd.concat([df2, df2_diverse])

In [39]:
df.groupby(['model', 'conversation','job','prompt_id','debiasing_id','gender']).sum(numeric_only=True).reset_index()

,model,conversation,job,prompt_id,debiasing_id,gender,pronoun_prob
0,mistral-7b,False,aircraft mechanic,1,0,diverse,0.162594
1,mistral-7b,False,aircraft mechanic,1,0,female,0.120473
2,mistral-7b,False,aircraft mechanic,1,0,male,0.084161
3,mistral-7b,False,brickmason,1,0,diverse,0.107949
4,mistral-7b,False,brickmason,1,0,female,0.101903
...,...,...,...,...,...,...,...
115,mistral-7b,False,vehicle technician,1,0,female,0.074626
116,mistral-7b,False,vehicle technician,1,0,male,0.412465
117,mistral-7b,False,vet,1,0,diverse,0.074673
118,mistral-7b,False,vet,1,0,female,0.142421
